<h2 style='color:purple' align='center'>Naive Bayes Tutorial Part 1: Predicting survival from titanic crash</h2>

In [2]:
import pandas as pd

In [10]:
df = pd.read_csv("titanic.csv")
df.head()


,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [9]:
df.shape

(891, 12)

In [14]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [15]:
inputs = df.drop('Survived',axis='columns') #x
target = df.Survived #y

In [16]:
#inputs.Sex = inputs.Sex.map({'male': 1, 'female': 2})

In [17]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [18]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


**I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female**

In [19]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [20]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [24]:
inputs.Age[:20]

0     22.0
1     38.0
2     26.0
3     35.0
4     35.0
5      NaN
6     54.0
7      2.0
8     27.0
9     14.0
10     4.0
11    58.0
12    20.0
13    39.0
14    14.0
15    55.0
16     2.0
17     NaN
18    31.0
19     NaN
Name: Age, dtype: float64

In [26]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head(10)

,Pclass,Age,Fare,female
0,3,22.000000,7.2500,0
1,1,38.000000,71.2833,1
2,3,26.000000,7.9250,1
3,1,35.000000,53.1000,1
4,3,35.000000,8.0500,0
5,3,29.699118,8.4583,0
6,1,54.000000,51.8625,0
7,3,2.000000,21.0750,0
8,3,27.000000,11.1333,1
9,2,14.000000,30.0708,1


In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [30]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [31]:
model.fit(X_train,y_train) # entrainement pour y et x train 

GaussianNB()

In [32]:
model.score(X_test,y_test) #le porcentage de reussite, evaluation du modele

0.75

In [33]:
X_test[0:10]

,Pclass,Age,Fare,female
450,2,36.000000,27.7500,0
416,2,34.000000,32.5000,1
348,3,3.000000,15.9000,0
829,1,62.000000,80.0000,1
707,1,42.000000,26.2875,0
720,2,6.000000,33.0000,1
556,1,48.000000,39.6000,1
437,2,24.000000,18.7500,1
466,2,29.699118,0.0000,0
357,2,38.000000,13.0000,1


In [17]:
y_test[0:10]

309    1
839    1
110    0
872    0
235    0
411    0
32     1
562    0
542    0
250    0
Name: Survived, dtype: int64

In [18]:
model.predict(X_test[0:10])

array([1, 0, 0, 0, 1, 0, 1, 0, 1, 0], dtype=int64)

In [34]:
model.predict_proba(X_test[:10]) # proba davoir 1 vs0

array([[0.93529537, 0.06470463],
       [0.14835393, 0.85164607],
       [0.93752042, 0.06247958],
       [0.00994074, 0.99005926],
       [0.80696022, 0.19303978],
       [0.08221436, 0.91778564],
       [0.04258308, 0.95741692],
       [0.14439402, 0.85560598],
       [0.93054186, 0.06945814],
       [0.16676158, 0.83323842]])

**Calculate the score using cross validation**

In [36]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5) #on calcule 5 fois le scor 

array([0.816     , 0.816     , 0.752     , 0.77419355, 0.75806452])

In [37]:
cross_val_score(GaussianNB(),X_test, y_test, cv=5) #on calcule 5 fois le scor 

array([0.77777778, 0.72222222, 0.81481481, 0.66037736, 0.69811321])

In [47]:
me_test=[[1,22,7.8,1]]
model.predict(me_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([1], dtype=int64)